# Проект для «Викишоп» с BERT

Цель проекта: построить модель классификации комментариев на позитивные и негативные. 

Заказчие: интернет-магазин «Викишоп».

Исходные данные: Набор данных с разметкой о токсичности правок. Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

Значение метрики качества *F1* не меньше 0.75. 

Зазачи:

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделайть выводы.


## Подготовка

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display


import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
import re

from tqdm import notebook

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


import torch
import transformers as ppb

### Загрузка данных

Загрузим данные и посмотрим их содержание

In [2]:
df = pd.read_csv('toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

В данных сильный дисбаланс. Токсичных коммантариев очень мало. 10% 

In [5]:
train, test = train_test_split(df, test_size=0.25, random_state=12345)

print('Обучающие объекты: {}'.format(train.shape))
print('Тестовые объекты: {}'.format(test.shape))

Обучающие объекты: (119678, 2)
Тестовые объекты: (39893, 2)


### Создадим признаки для обучения с использованием TF-IDF

Создадим корпус: преобразуем столбец text в список текстов.

In [6]:
corpus_train = train['text'].reset_index(drop=True)
corpus_test = test['text'].reset_index(drop=True)

In [7]:
# Токинезируем и лемматизируем
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    m = WordNetLemmatizer()
    lemm_list = [m.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    lemm_text = " ".join(lemm_list)
    return lemm_text

In [8]:
#Чистим текст от лишних символов используя регулярные выражения
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.split()
    text = " ".join(text)
    return text

Проверим работу очистки и лемматизации на примере одного комментария.

In [9]:
corpus_train[2]

'":::We are in the same boat as Britannica (which is actually published in the States). Britannica is just one source though, albeit a very weighty one (literally!). N466 \n"'

In [10]:
%%time
lemmatize(clear_text(corpus_train[2]))

CPU times: total: 2.28 s
Wall time: 2.3 s


'We be in the same boat a Britannica which be actually publish in the States Britannica be just one source though albeit a very weighty one literally N'

Лемматизируем корпус. Используем библиатеку tqdm для отображения выполения процесса.

In [11]:
%%time
lemm_text_train = []
for i in notebook.tqdm(range(len(corpus_train))):
    lemm_text_train.append(lemmatize(clear_text(corpus_train[i])))

  0%|          | 0/119678 [00:00<?, ?it/s]

CPU times: total: 1h 15min 22s
Wall time: 1h 15min 11s


In [12]:
%%time
lemm_text_test = []
for i in notebook.tqdm(range(len(corpus_test))):
    lemm_text_test.append(lemmatize(clear_text(corpus_test[i])))

  0%|          | 0/39893 [00:00<?, ?it/s]

CPU times: total: 24min 38s
Wall time: 24min 35s


Создадим признаки для обучения моделей классификации на основе оценки важности слов TF-IDF

In [13]:
#Вычислим TF-IDF для лемматизорованного корпуса текстов.
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
count_tf_idf.fit(lemm_text_train)                    #учим на train
features_train_T = count_tf_idf.transform(lemm_text_train)
features_test_T = count_tf_idf.transform(lemm_text_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Electronic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Выделим целевой признак

In [14]:
target_train_T = train['toxic']
target_test_T = test['toxic']

### Создадим признаки для обучения с BERT

Выберем устройтсво для вычисления: Процессор или видеокарта

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
torch.cuda.get_device_name(0)

'Quadro M1000M'

Из библиотеки transformers загрузим обученную модель DistilBERT и токенайзер. DistilBERT - облегченная модель BERT

In [17]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Токенизируем текст с добавлением токенов начала и конца предложения с ограниченим длинны предложения до 256 слов. Можно ограничивать до 512, но вычисления гораздо дольше.

In [18]:
%%time
tokenized_train = train['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=256, truncation=True)))

CPU times: total: 4min 9s
Wall time: 4min 10s


In [19]:
%%time
tokenized_test = test['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=256, truncation=True)))

CPU times: total: 1min 21s
Wall time: 1min 21s


In [20]:
tokenized_train

111565    [101, 1000, 2012, 2232, 18856, 2401, 2705, 293...
8575      [101, 1000, 2469, 2518, 1012, 2011, 1996, 2126...
153402    [101, 1000, 1024, 1024, 1024, 2057, 2024, 1999...
65019     [101, 1000, 1045, 2018, 1037, 2298, 2083, 1996...
155787    [101, 1000, 6031, 3222, 8327, 6356, 2575, 1011...
                                ...                        
109993    [101, 11721, 2213, 1010, 2562, 2115, 10231, 21...
85412     [101, 1045, 13371, 2054, 2017, 3205, 1012, 101...
133249    [101, 1000, 17012, 1010, 2748, 1010, 2004, 104...
130333      [101, 17704, 2066, 8764, 7507, 2884, 4027, 102]
77285     [101, 9181, 2005, 3972, 20624, 2239, 1045, 100...
Name: text, Length: 119678, dtype: object

In [21]:
tokenized_test

146790    [101, 6289, 2232, 3844, 1996, 6616, 2039, 2017...
2941      [101, 1000, 7514, 1024, 2045, 2003, 2053, 2107...
115087    [101, 7514, 4931, 1010, 2017, 2071, 2012, 2560...
48830     [101, 2008, 2015, 2986, 1010, 2045, 2003, 2053...
136034    [101, 1000, 1040, 15922, 6488, 1997, 2442, 540...
                                ...                        
18617     [101, 2748, 2909, 1010, 2720, 1056, 24281, 101...
69106     [101, 2204, 6998, 1012, 2085, 2123, 1005, 1056...
127941    [101, 2748, 1012, 11701, 1012, 2096, 7987, 201...
98965     [101, 3531, 2644, 1012, 2065, 2017, 3613, 2000...
36200     [101, 1000, 2092, 1010, 1996, 3720, 2003, 2494...
Name: text, Length: 39893, dtype: object

Приведем векторы к одному размеру путем прибавления к более коротким векторам идентификатора 0 (padding)

In [22]:
max_len = 0
for i in tokenized_train.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

256
256


In [23]:
padded_train

array([[  101,  1000,  2012, ...,     0,     0,     0],
       [  101,  1000,  2469, ...,     0,     0,     0],
       [  101,  1000,  1024, ...,     0,     0,     0],
       ...,
       [  101,  1000, 17012, ...,     0,     0,     0],
       [  101, 17704,  2066, ...,     0,     0,     0],
       [  101,  9181,  2005, ...,     0,     0,     0]])

In [24]:
padded_test

array([[ 101, 6289, 2232, ...,    0,    0,    0],
       [ 101, 1000, 7514, ...,    0,    0,    0],
       [ 101, 7514, 4931, ...,    0,    0,    0],
       ...,
       [ 101, 2748, 1012, ...,    0,    0,    0],
       [ 101, 3531, 2644, ...,    0,    0,    0],
       [ 101, 1000, 2092, ...,    0,    0,    0]])

In [25]:
padded_train.shape

(119678, 256)

In [26]:
padded_test.shape

(39893, 256)

Отбросим добавленные нулевые токены, чтобы модель не обращала на них внимания.

In [27]:
attention_mask_train = np.where(padded_train != 0, 1, 0)
attention_mask_train.shape

(119678, 256)

In [28]:
attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

(39893, 256)

Считаем эмбэдинги батчами. Размер батча подбираем таким, чтобы уместится в оперативной памяти видеокарты, если считаем на ней или в оперативной памяти комьютера, если считаем на процессоре. Используем библиатеку tqdm для отображения выполения процесса.

In [29]:
%%time
batch_size = 20
embeddings = []
for i in notebook.tqdm(range(padded_train.shape[0] // batch_size)):
    batch = torch.tensor(padded_train[batch_size*i:batch_size*(i+1)], device=device)
    attention_mask_batch = torch.tensor(attention_mask_train[batch_size*i:batch_size*(i+1)], device=device)
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    
    del batch
    del attention_mask_batch
    del batch_embeddings
features_train_B = np.concatenate(embeddings)
del embeddings

  0%|          | 0/5983 [00:00<?, ?it/s]

CPU times: total: 1h 10min 25s
Wall time: 1h 10min 27s


In [30]:
display(features_train_B)

array([[-0.13084534, -0.01765354, -0.18243226, ..., -0.1501868 ,
         0.29446858,  0.40739706],
       [ 0.05222206, -0.22805226, -0.0042829 , ...,  0.00651409,
         0.31771728,  0.3818135 ],
       [-0.07801925, -0.06811096,  0.0301069 , ..., -0.10018516,
         0.6366417 ,  0.49522316],
       ...,
       [-0.05477807,  0.13115141, -0.3384153 , ..., -0.10216154,
         0.572422  ,  0.4058672 ],
       [ 0.02020259, -0.05602971,  0.1558377 , ...,  0.02143765,
         0.3102176 ,  0.18747449],
       [ 0.12565956, -0.04043335, -0.31949914, ..., -0.05296273,
         0.43155682,  0.2975121 ]], dtype=float32)

In [31]:
%%time
batch_size = 20
embeddings = []
for i in notebook.tqdm(range(padded_test.shape[0] // batch_size)):
    batch = torch.tensor(padded_test[batch_size*i:batch_size*(i+1)], device=device)
    attention_mask_batch = torch.tensor(attention_mask_test[batch_size*i:batch_size*(i+1)], device=device)
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    
    del batch
    del attention_mask_batch
    del batch_embeddings
features_test_B = np.concatenate(embeddings)
del embeddings

  0%|          | 0/1994 [00:00<?, ?it/s]

CPU times: total: 23min 25s
Wall time: 23min 26s


In [32]:
display(features_test_B)

array([[ 0.23546158,  0.35464013,  0.07846079, ..., -0.22472392,
         0.5638445 ,  0.17181668],
       [ 0.07774356,  0.08934345, -0.30955186, ..., -0.00533941,
         0.4839362 ,  0.45248052],
       [ 0.19198923,  0.03568441, -0.02476341, ..., -0.04271309,
         0.5287502 ,  0.31306303],
       ...,
       [ 0.06803254, -0.08022261, -0.12355368, ...,  0.02971786,
         0.23437576,  0.43523535],
       [-0.23212557, -0.08912389, -0.55926144, ..., -0.26671115,
         0.19803935,  0.08390954],
       [-0.08286869,  0.01685899, -0.03421983, ..., -0.0522169 ,
         0.3505435 ,  0.45877573]], dtype=float32)

Выделим целевой признак и состыкуем размеры выборок. Иногда при батчировании несколько строк пропадает.

In [33]:
target_train_B = train['toxic'][0:features_train_B.shape[0]]
target_test_B = test['toxic'][0:features_test_B.shape[0]]

## Обучение

### Обучение логистической регрессии на признаках TF-IDF

Учим логистическую регрессию

In [118]:
%%time
model_LR_T = LogisticRegression(C=10, random_state=12345, solver='liblinear')
model_LR_T.fit(features_train_T, target_train_T)

CPU times: total: 6.98 s
Wall time: 1.81 s


LogisticRegression(C=10, random_state=12345, solver='liblinear')

Посчитаем метрику F1 на тренировочной выборке

In [119]:
F1_LR_T_train = f1_score(target_train_T, model_LR_T.predict(features_train_T))
F1_LR_T_train

0.9092888559857161

Посчитаем метрику F1 на тестовой выборке

In [120]:
F1_LR_T_test = f1_score(target_test_T, model_LR_T.predict(features_test_T))
F1_LR_T_test

0.780812610409023

### Обучение логистической регрессии на признаках BERT

Учим логистическую регрессию

In [132]:
%%time

model_LR_B = LogisticRegression(C=20,random_state=12345, solver='liblinear')
model_LR_B.fit(features_train_B, target_train_B)

CPU times: total: 1min 11s
Wall time: 1min 11s


LogisticRegression(C=20, random_state=12345, solver='liblinear')

Посчитаем метрику F1 на тренировочной выборке

In [133]:
F1_LR_B_train = f1_score(target_train_B, model_LR_B.predict(features_train_B))
F1_LR_B_train

0.7557864042281757

Посчитаем метрику F1 на тестовой выборке

In [134]:
F1_LR_B_test = f1_score(target_test_B, model_LR_B.predict(features_test_B))
F1_LR_B_test

0.7457673032642558

## Выводы

In [135]:
pd.DataFrame({'Модель и способ векторизации': ['Логистическая регрессия c TF-IDF', 'Логистическая регрессия c BERT'], 
              'F1 train': [F1_LR_T_train, F1_LR_B_train],
              'F1 test': [F1_LR_T_test, F1_LR_B_test]})

,Модель и способ векторизации,F1 train,F1 test
0,Логистическая регрессия c TF-IDF,0.909289,0.780813
1,Логистическая регрессия c BERT,0.755786,0.745767


Построены 2 модели классификации коментариев на позитивные и негативные. В качестве можели классификации использована логистическая регрессия. Для подготовки признаков (векторизации текста) использова лись 2 метода: первый основан на лемматизации и оценки важности слов TF-IDF, второй  использовал обученную нейросеть BERT. Метриики сильно зависят от размера обучающих данных. Чтоб получить заданные метрики пришлось использовать весь датасет. Балансировка классов downsample или class_weight='balanced' понижает метрики. Подбор параметров кроссвалидацией позворляет немного улучшить метрики, но на таком объема данных занимает очень много времени. Модель логистической регрессии с TF-IDF признаками показала более высокие метрики чем модель BERT признаками.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны